# Data Cleaning
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/11-7tK27HdchlFw8lEj7NaJBu-IJ4NnwJ)  
**Step 1: Handling missing values**  
1. Remove records with NAs
2. Remove 'undefined' value

**Step 2: Standardize mixed data formats**  
1. Date format  
2. Time format

**Step 3: Prepare date for visualization**
1. Data modification
2. Handling remaining errors


In [ ]:
# import libraries
import pandas as pd
import numpy as np
import datetime
from datetime import date

In [ ]:
# import dataset
df_activity = pd.read_csv("user_activity_draft.csv", delimiter=';', usecols = ['user_activity','email', 'date', 'time'])
df_activity

,user_activity,email,date,time
0,article_access,useremail219@gmail.com,03/02/2021,21:19
1,article_access,useremail107@gmail.com,03/02/2021,23:13
2,article_access,useremail316@gmail.com,03/03/2021,06:32
3,article_access,useremail170@bendigo.vic.gov.au,03/03/2021,08:51
4,article_access,useremail214@sydney.edu.au,03/03/2021,09:19
...,...,...,...,...
60204,foreground,undefined,undefined,undefined
60205,foreground,undefined,undefined,undefined
60206,foreground,undefined,undefined,undefined
60207,foreground,undefined,undefined,undefined


In [ ]:
# Check for NA values
df_activity.isnull().sum()

user_activity       0
email            5144
date                0
time                0
dtype: int64

In [ ]:
# Remove NA values
df_activity_dropna = df_activity.dropna()
print(df_activity.shape[0] - df_activity_dropna.shape[0], ' records has been removed')

5144  records has been removed


In [ ]:
# Remove 'undefined' email value
df_activity_removed = df_activity_dropna[(df_activity_dropna['email'] != 'undefined')]
print(df_activity_dropna.shape[0] - df_activity_removed.shape[0], ' undefined records has been removed')

8797  undefined records has been removed


In [ ]:
# Return df
df_activity_removed

,user_activity,email,date,time
0,article_access,useremail219@gmail.com,03/02/2021,21:19
1,article_access,useremail107@gmail.com,03/02/2021,23:13
2,article_access,useremail316@gmail.com,03/03/2021,06:32
3,article_access,useremail170@bendigo.vic.gov.au,03/03/2021,08:51
4,article_access,useremail214@sydney.edu.au,03/03/2021,09:19
...,...,...,...,...
59854,foreground,useremail37@hotmail.com,12/04/2021,4:04 PM
59855,foreground,useremail318@gmail.com,12/04/2021,4:36 PM
59856,foreground,useremail8@mybabynow.org.au,4/13/2021,10:13 AM
59857,foreground,useremail369@gmail.com,13/04/2021,10:19 AM


## Handling date data
Convert date data to one format YY-MM-DD because they are in mixed formats

In [ ]:
# define function to handling date
def convert_date(date_text):
  date_formatted = None
  try:
      date_formatted = datetime.datetime.strptime(date_text, '%d/%m/%Y')
  except ValueError:
      print('.')

  if date_formatted == None:
      try:
          date_formatted = datetime.datetime.strptime(date_text, '%m/%d/%Y')
      except ValueError:
          print('.')

  if date_formatted == None:
      try:
          date_formatted = datetime.datetime.strptime(date_text, '%Y/%m/%d')
      except ValueError:
          print('.')

  if date_formatted == None:
      try:
          date_formatted = datetime.datetime.strptime(date_text, '%d.%m.%Y')
      except ValueError:
          print('.')

  if date_formatted == None:
      try:
          date_formatted = datetime.datetime.strptime(date_text, '%Y. %m. %d.')
      except ValueError:
          print('.')

  if date_formatted == None:
      try:
          date_formatted = datetime.datetime.strptime(date_text, '%Y-%m-%d')
      except ValueError:
          print('.')

  if date_formatted == None:
      try:
          date_formatted = datetime.datetime.strptime(date_text, '%d-%m-%Y')
      except ValueError:
          print('.')

  return date_formatted

In [ ]:
# Create date_validate for new date value
df_activity_removed['date_validate'] = df_activity_removed['date'].apply(convert_date)

In [ ]:
df_activity_removed

,user_activity,email,date,time,date_validate
0,article_access,useremail219@gmail.com,03/02/2021,21:19,2021-02-03
1,article_access,useremail107@gmail.com,03/02/2021,23:13,2021-02-03
2,article_access,useremail316@gmail.com,03/03/2021,06:32,2021-03-03
3,article_access,useremail170@bendigo.vic.gov.au,03/03/2021,08:51,2021-03-03
4,article_access,useremail214@sydney.edu.au,03/03/2021,09:19,2021-03-03
...,...,...,...,...,...
59854,foreground,useremail37@hotmail.com,12/04/2021,4:04 PM,2021-04-12
59855,foreground,useremail318@gmail.com,12/04/2021,4:36 PM,2021-04-12
59856,foreground,useremail8@mybabynow.org.au,4/13/2021,10:13 AM,2021-04-13
59857,foreground,useremail369@gmail.com,13/04/2021,10:19 AM,2021-04-13


## Handling time data
Convert time data to one format 24H because they are in mixed formats

In [ ]:
# Define fucntion for handling time
def unify_time(input):
  if 'AM' in input or 'PM' in input or 'p.m.' in input or 'a.m.' in input or 'a. m.' in input:
    if 'a. m.' in input:
      input = input.split('a. m.')[0]

    time = input.split(' ')[0]
    if 'PM' or 'p.m.' in input:
        time_seg = time.split(':')
        if int(time_seg[0]) < 12:
          return (str(int(time_seg[0]) + 12) + ':' + time_seg[1])
        else:
          return (time_seg[0] + ':' + time_seg[1])
    else:
      return time
  else:
    return input

In [ ]:
df_activity_removed['time_validate'] = df_activity_removed['time'].apply(unify_time)
df_activity_removed

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,user_activity,email,date,time,date_validate,time_validate
0,article_access,useremail219@gmail.com,03/02/2021,21:19,2021-02-03,21:19
1,article_access,useremail107@gmail.com,03/02/2021,23:13,2021-02-03,23:13
2,article_access,useremail316@gmail.com,03/03/2021,06:32,2021-03-03,06:32
3,article_access,useremail170@bendigo.vic.gov.au,03/03/2021,08:51,2021-03-03,08:51
4,article_access,useremail214@sydney.edu.au,03/03/2021,09:19,2021-03-03,09:19
...,...,...,...,...,...,...
59854,foreground,useremail37@hotmail.com,12/04/2021,4:04 PM,2021-04-12,16:04
59855,foreground,useremail318@gmail.com,12/04/2021,4:36 PM,2021-04-12,16:36
59856,foreground,useremail8@mybabynow.org.au,4/13/2021,10:13 AM,2021-04-13,22:13
59857,foreground,useremail369@gmail.com,13/04/2021,10:19 AM,2021-04-13,22:19


## Check data validation

In [ ]:
# Future date
df_activity_removed[df_activity_removed['date_validate'] > pd.to_datetime(date.today())]

,user_activity,email,date,time,date_validate,time_validate
233,article_access,useremail1@gmail.com,03/08/2021,09:29,2021-08-03,09:29
234,article_access,useremail279@gleneira.vic.gov.au,03/08/2021,10:35,2021-08-03,10:35
235,article_access,useremail279@gleneira.vic.gov.au,03/08/2021,10:35,2021-08-03,10:35
236,article_access,useremail312@hotmail.com,03/08/2021,10:26,2021-08-03,10:26
237,article_access,useremail312@hotmail.com,03/08/2021,10:27,2021-08-03,10:27
...,...,...,...,...,...,...
59759,foreground,useremail490@gmail.com,4/11/2021,12:06 PM,2021-11-04,12:06
59760,foreground,useremail465@y7mail.com,4/12/2021,9:50 AM,2021-12-04,21:50
59761,foreground,useremail190@gmail.com,4/12/2021,4:27 PM,2021-12-04,16:27
59850,foreground,useremail8@mybabynow.org.au,4/10/2021,4:54 PM,2021-10-04,16:54


In [ ]:
# Remove future date
df_activity_current = df_activity_removed[df_activity_removed['date_validate'] < pd.to_datetime(date.today())]
print(df_activity_removed.shape[0] - df_activity_current.shape[0], ' records has been removed')
df_activity_current

3949  records has been removed


,user_activity,email,date,time,date_validate,time_validate
0,article_access,useremail219@gmail.com,03/02/2021,21:19,2021-02-03,21:19
1,article_access,useremail107@gmail.com,03/02/2021,23:13,2021-02-03,23:13
2,article_access,useremail316@gmail.com,03/03/2021,06:32,2021-03-03,06:32
3,article_access,useremail170@bendigo.vic.gov.au,03/03/2021,08:51,2021-03-03,08:51
4,article_access,useremail214@sydney.edu.au,03/03/2021,09:19,2021-03-03,09:19
...,...,...,...,...,...,...
59854,foreground,useremail37@hotmail.com,12/04/2021,4:04 PM,2021-04-12,16:04
59855,foreground,useremail318@gmail.com,12/04/2021,4:36 PM,2021-04-12,16:36
59856,foreground,useremail8@mybabynow.org.au,4/13/2021,10:13 AM,2021-04-13,22:13
59857,foreground,useremail369@gmail.com,13/04/2021,10:19 AM,2021-04-13,22:19


## Merge with group list

In [ ]:
df_grouplist = pd.read_csv("user_grouplist.csv", delimiter=';', usecols = ['user_group','email'])
df_grouplist

,user_group,email
0,antenatal,useremail1@gmail.com
1,antenatal,useremail2@gmail.com
2,antenatal,useremail3@hotmail.com
3,antenatal,useremail4@outlook.com
4,antenatal,useremail5@hotmail.com
...,...,...
494,clinician,useremail495@gmail.com
495,antenatal,useremail496@hotmail.com
496,clinician,useremail497@hotmail.com
497,antenatal,useremail498@gmail.com


In [ ]:
# Merge with email list table to match with the user group
df_activity_merge = pd.merge(df_activity_current, df_grouplist, left_on="email", right_on="email")
df_activity_merge

,user_activity,email,date,time,date_validate,time_validate,user_group
0,article_access,useremail219@gmail.com,03/02/2021,21:19,2021-02-03,21:19,clinician
1,article_access,useremail219@gmail.com,03/02/2021,21:20,2021-02-03,21:20,clinician
2,article_access,useremail219@gmail.com,03/02/2021,21:20,2021-02-03,21:20,clinician
3,article_access,useremail219@gmail.com,03/02/2021,21:19,2021-02-03,21:19,clinician
4,article_access,useremail219@gmail.com,03/02/2021,21:19,2021-02-03,21:19,clinician
...,...,...,...,...,...,...,...
42314,terminate,useremail405@hotmail.com,9/04/2021,8:28 AM,2021-04-09,20:28,clinician
42315,login,useremail260@yahoo.com,02/27/2021,23:00,2021-02-27,23:00,clinician
42316,login,useremail239@yarraranges.vic.gov.au,02/16/2021,14:32,2021-02-16,14:32,clinician
42317,terminate,useremail239@yarraranges.vic.gov.au,16/02/2021,2:34 PM,2021-02-16,14:34,clinician


In [ ]:
# Before join with group list
print(df_activity_current['email'].nunique(), 'unique email before join with group list')
print(df_activity_merge['email'].nunique(), 'unique email remain after join with group list')

496 unique email before join with group list
496 unique email remain after join with group list


## Convert time to 2400
Convert time to 2400 for visualization

In [ ]:
# Remove ":" and "?" from time
df_activity_merge['time_validate'] = df_activity_merge['time_validate'].str.replace(':', '')
df_activity_merge['time_validate'] = df_activity_merge['time_validate'].str.replace('?', '')
df_activity_merge

,user_activity,email,date,time,date_validate,time_validate,user_group
0,article_access,useremail219@gmail.com,03/02/2021,21:19,2021-02-03,2119,clinician
1,article_access,useremail219@gmail.com,03/02/2021,21:20,2021-02-03,2120,clinician
2,article_access,useremail219@gmail.com,03/02/2021,21:20,2021-02-03,2120,clinician
3,article_access,useremail219@gmail.com,03/02/2021,21:19,2021-02-03,2119,clinician
4,article_access,useremail219@gmail.com,03/02/2021,21:19,2021-02-03,2119,clinician
...,...,...,...,...,...,...,...
42314,terminate,useremail405@hotmail.com,9/04/2021,8:28 AM,2021-04-09,2028,clinician
42315,login,useremail260@yahoo.com,02/27/2021,23:00,2021-02-27,2300,clinician
42316,login,useremail239@yarraranges.vic.gov.au,02/16/2021,14:32,2021-02-16,1432,clinician
42317,terminate,useremail239@yarraranges.vic.gov.au,16/02/2021,2:34 PM,2021-02-16,1434,clinician


In [ ]:
# double check 
df_activity_merge.sort_values('time',ascending=False)

,user_activity,email,date,time,date_validate,time_validate,user_group
29333,topic_access,useremail483@mitchellshire.vic.gov.au,11/10/2020,?10?:?17,2020-10-11,1017,clinician
29330,article_access,useremail483@mitchellshire.vic.gov.au,11/10/2020,?10?:?17,2020-10-11,1017,clinician
29334,topic_access,useremail483@mitchellshire.vic.gov.au,11/10/2020,?10?:?17,2020-10-11,1017,clinician
29331,tool_access,useremail483@mitchellshire.vic.gov.au,11/10/2020,?10?:?14,2020-10-11,1014,clinician
29332,topic_access,useremail483@mitchellshire.vic.gov.au,11/10/2020,?10?:?14,2020-10-11,1014,clinician
...,...,...,...,...,...,...,...
37975,login,useremail492@hotmail.com,04/01/2021,00:07,2021-01-04,0007,antenatal
13111,forum_access,useremail282@me.com,03/25/2021,00:07,2021-03-25,0007,clinician
15078,topic_access,useremail282@me.com,03/25/2021,00:07,2021-03-25,0007,clinician
17774,login,useremail282@me.com,03/25/2021,00:06,2021-03-25,0006,clinician


## Modify any time >2400 to 0000
Time error with more than 2400, need to resolve

In [ ]:
## Convert to int
df_activity_modify = df_activity_merge
df_activity_modify['time_validate']= df_activity_modify['time_validate'].astype(int)

def standard_time(input):
  if input > 2400:
    return (input - 2400)
  else:
    return input
df_activity_modify['time_validate'] = df_activity_modify['time_validate'].apply(standard_time)
df_activity_modify

,user_activity,email,date,time,date_validate,time_validate,user_group
0,article_access,useremail219@gmail.com,03/02/2021,21:19,2021-02-03,2119,clinician
1,article_access,useremail219@gmail.com,03/02/2021,21:20,2021-02-03,2120,clinician
2,article_access,useremail219@gmail.com,03/02/2021,21:20,2021-02-03,2120,clinician
3,article_access,useremail219@gmail.com,03/02/2021,21:19,2021-02-03,2119,clinician
4,article_access,useremail219@gmail.com,03/02/2021,21:19,2021-02-03,2119,clinician
...,...,...,...,...,...,...,...
42314,terminate,useremail405@hotmail.com,9/04/2021,8:28 AM,2021-04-09,2028,clinician
42315,login,useremail260@yahoo.com,02/27/2021,23:00,2021-02-27,2300,clinician
42316,login,useremail239@yarraranges.vic.gov.au,02/16/2021,14:32,2021-02-16,1432,clinician
42317,terminate,useremail239@yarraranges.vic.gov.au,16/02/2021,2:34 PM,2021-02-16,1434,clinician


In [ ]:
# 0 value more than 2400
df_activity_modify[df_activity_modify["time_validate"]> 2400]

,user_activity,email,date,time,date_validate,time_validate,user_group


In [ ]:
# Final dataset
df_activity_clean = df_activity_modify[['user_group', 'user_activity', 'email','date_validate', 'time_validate']]
df_activity_clean

,user_group,user_activity,email,date_validate,time_validate
0,clinician,article_access,useremail219@gmail.com,2021-02-03,2119
1,clinician,article_access,useremail219@gmail.com,2021-02-03,2120
2,clinician,article_access,useremail219@gmail.com,2021-02-03,2120
3,clinician,article_access,useremail219@gmail.com,2021-02-03,2119
4,clinician,article_access,useremail219@gmail.com,2021-02-03,2119
...,...,...,...,...,...
42314,clinician,terminate,useremail405@hotmail.com,2021-04-09,2028
42315,clinician,login,useremail260@yahoo.com,2021-02-27,2300
42316,clinician,login,useremail239@yarraranges.vic.gov.au,2021-02-16,1432
42317,clinician,terminate,useremail239@yarraranges.vic.gov.au,2021-02-16,1434


## Export file

In [ ]:
from google.colab import files
df_activity_clean.to_csv('user_activity_clean.csv') 
files.download('user_activity_clean.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>